In [ ]:
from __future__ import absolute_import, division, print_function

from hippylib import nb
import dolfin as dl
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
%matplotlib inline

import logging
logging.getLogger('FFC').setLevel(logging.ERROR)
logging.getLogger('UFC').setLevel(logging.ERROR)
dl.set_log_active(False)

sys.path.insert(0,'/home/fenics/Installations/MUQ_INSTALL/lib')
from IPython.display import Image

# MUQ Includes
import pymuqModeling as mm # Needed for Gaussian distribution
import pymuqApproximation as ma # Needed for Gaussian processes
import pymuqSamplingAlgorithms as ms # Needed for MCMC

from PAT_forward import PAT_forward

# Hyperparameters

In [ ]:
nx = ny = 10
mesh = dl.RectangleMesh(dl.Point(0, 0),dl.Point(1,1),nx,ny,"right")
V = dl.FunctionSpace(mesh, 'P', 1)
time_final = 9.5e-6
numSteps = 14
numObs = 2*(nx-1)+2*(nx-3)
c = 1500

# Find Matrix

In [ ]:
fwdSolver = PAT_forward(time_final, numSteps, c, V, numObs, nx, True)


m = dl.interpolate(dl.Expression('0.5 * exp(-(x[0]-0.5)*(x[0]-0.5)/0.05 -(x[1]-0.5)*(x[1]-0.5)/0.05)', degree=2), V)

# Observation data
obsData = fwdSolver.Evaluate([m.vector().get_local()])[0]

size = obsData.shape[0]

I = np.identity(V.dim())

K = np.zeros((size,V.dim()))
# Multiply forward solver and columns of I to build K Matrix 
for i in range(V.dim()):
    K[:,i] = fwdSolver.Evaluate([I[:,i]])[0]



# eig_K = np.linalg.eigvals(K)
# eig_K = sorted(eig_K,reverse=True)

# normalized_eig_K = eig_K/eig_K[0]

# print('The condtiond number of K is ', np.linalg.cond(K))

# plt.figure()
# plt.semilogy(eig_K )
# plt.title('Eigvals')

# plt.figure()
# plt.semilogy(normalized_eig_K)
# plt.title('Normalized Eigvals')



In [ ]:
d = np.dot( K , m.vector().get_local())
print('The norm of obsData - d is ', np.linalg.norm(d-obsData))

In [ ]:
type(m.vector().get_local())

In [ ]:
d.shape

In [ ]:
K.shape